In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import math
import json
import boto3
import sagemaker

In [2]:
bucket = 'cic-network-data'

In [3]:
data = 'abt.csv'

In [4]:
s3 = boto3.resource('s3')

In [5]:
role = sagemaker.get_execution_role()

In [6]:
s3.Bucket(bucket).download_file(data, 'abt.csv')

In [7]:
df = pd.read_csv('./abt.csv')

In [8]:
from sklearn.datasets import dump_svmlight_file

In [9]:
df.shape

(16232943, 77)

In [10]:
df.isnull().sum().sort_values(ascending=False)

Flow Pkts/s         95760
Flow Byts/s         36039
Year                    0
Flow IAT Std            0
Flow IAT Min            0
Fwd IAT Tot             0
Fwd IAT Mean            0
Fwd IAT Std             0
Fwd IAT Max             0
Fwd IAT Min             0
Bwd IAT Tot             0
Bwd IAT Mean            0
Bwd IAT Std             0
Bwd IAT Max             0
Bwd IAT Min             0
Fwd PSH Flags           0
Fwd URG Flags           0
Fwd Header Len          0
Bwd Header Len          0
Flow IAT Max            0
Flow IAT Mean           0
Bwd Pkts/s              0
TotLen Bwd Pkts         0
Dst Port                0
Protocol                0
Flow Duration           0
Tot Fwd Pkts            0
Tot Bwd Pkts            0
TotLen Fwd Pkts         0
Fwd Pkt Len Max         0
                    ...  
Active Max              0
Idle Mean               0
Pkt Len Max             0
Idle Std                0
Idle Max                0
Idle Min                0
Hour                    0
Minute      

In [11]:
bad_vals = df.isin([np.nan, np.inf, -np.inf])


In [12]:
df[bad_vals].head()

,Label,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Idle Mean,Idle Std,Idle Max,Idle Min,Hour,Minute,Second,Day,Month,Year
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
neg = df['Flow Byts/s'].isin([np.nan, np.inf, -np.inf])

In [14]:
df[neg].shape

(0, 77)

In [15]:
df.fillna(0, inplace=True)

In [45]:
df.iloc[:,0:10].describe()

,Label,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min
count,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07
mean,1.692999e-01,9.164073e+03,8.754113e+00,1.181380e+07,2.353319e+01,6.312706e+00,9.730373e+02,4.730931e+03,2.007620e+02,1.107600e+01
std,3.750166e-01,1.892224e+04,4.920117e+00,4.940124e+08,1.521137e+03,1.640256e+02,6.216906e+04,2.344659e+05,3.035091e+02,2.422938e+01
min,0.000000e+00,0.000000e+00,0.000000e+00,-9.190110e+11,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,5.300000e+01,6.000000e+00,4.920000e+02,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,8.000000e+01,6.000000e+00,2.097000e+04,2.000000e+00,1.000000e+00,4.300000e+01,1.010000e+02,4.000000e+01,0.000000e+00
75%,0.000000e+00,3.389000e+03,1.700000e+01,3.002116e+06,5.000000e+00,4.000000e+00,4.560000e+02,5.820000e+02,3.080000e+02,2.900000e+01
max,1.000000e+00,6.553500e+04,1.700000e+01,1.200000e+08,3.096290e+05,1.231180e+05,1.443918e+08,1.563604e+08,6.444000e+04,1.460000e+03


In [46]:
df.iloc[:,10:20].describe()

,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,Bwd Pkt Len Min,Bwd Pkt Len Mean,Bwd Pkt Len Std,Flow Byts/s,Flow Pkts/s,Flow IAT Mean,Flow IAT Std
count,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07
mean,5.031274e+01,7.084147e+01,3.501495e+02,2.647805e+01,1.130759e+02,1.315657e+02,2.555186e+05,5.198848e+04,3.335397e+06,1.275574e+06
std,6.050561e+01,1.164407e+02,4.974234e+02,5.097873e+01,1.641056e+02,2.042592e+02,3.657059e+06,2.639734e+05,2.229483e+08,3.381128e+08
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-8.895325e-03,-8.282200e+11,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.874731e+00,3.220000e+02,0.000000e+00
50%,3.600000e+01,0.000000e+00,9.500000e+01,0.000000e+00,6.600000e+01,0.000000e+00,7.759856e+02,1.312508e+02,1.171400e+04,6.065311e+01
75%,6.740000e+01,1.046196e+02,4.880000e+02,4.800000e+01,1.625000e+02,2.404163e+02,6.758833e+04,5.444646e+03,2.187450e+05,2.457262e+05
max,1.652931e+04,1.840158e+04,6.516000e+04,1.460000e+03,3.387928e+04,2.244841e+04,1.806643e+09,6.000000e+06,1.200000e+08,4.743545e+11


In [47]:
df.iloc[:,20:30].describe()

,Flow IAT Max,Flow IAT Min,Fwd IAT Tot,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Tot,Bwd IAT Mean,Bwd IAT Std
count,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07
mean,6.602697e+06,2.321854e+06,1.152313e+07,3.661300e+06,1.402965e+06,6.409625e+06,2.409649e+06,7.597374e+06,8.241998e+05,8.511741e+05
std,6.595104e+08,7.476211e+08,4.940071e+08,2.229602e+08,3.381193e+08,6.595084e+08,7.476241e+08,2.586288e+07,4.340497e+06,3.376930e+06
min,-8.282200e+11,-9.474050e+11,-9.190110e+11,-8.282200e+11,0.000000e+00,-8.282200e+11,-9.474050e+11,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.930000e+02,5.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.870000e+04,5.300000e+01,5.140000e+03,3.600750e+03,0.000000e+00,5.029000e+03,3.700000e+01,0.000000e+00,0.000000e+00,0.000000e+00
75%,1.070268e+06,9.270000e+02,2.343340e+06,3.197041e+05,1.480629e+05,1.177903e+06,6.540000e+02,5.510835e+05,1.643760e+05,3.794618e+04
max,9.797810e+11,1.200000e+08,1.200000e+08,1.200000e+08,4.743545e+11,9.797810e+11,1.200000e+08,1.200000e+08,1.200000e+08,8.483719e+07


In [48]:
df.iloc[:,30:40].describe()

,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Len,Bwd Header Len,Fwd Pkts/s,Bwd Pkts/s,Pkt Len Min,Pkt Len Max
count,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07
mean,2.612947e+06,2.912304e+05,4.389543e-02,1.638027e-04,2.575408e+02,1.330935e+02,3.667251e+04,1.531596e+04,1.094256e+01,3.893461e+02
std,1.024426e+07,3.825444e+06,2.048624e-01,1.279750e-02,1.226352e+04,3.265496e+03,2.133241e+05,9.241990e+04,2.261941e+01,5.141581e+02
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.600000e+01,0.000000e+00,2.548888e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.000000e+01,1.600000e+01,7.393169e+01,3.696231e+00,0.000000e+00,9.700000e+01
75%,2.181700e+05,1.570000e+02,0.000000e+00,0.000000e+00,1.320000e+02,1.040000e+02,3.164557e+03,7.911392e+02,2.900000e+01,9.350000e+02
max,1.200000e+08,1.200000e+08,1.000000e+00,1.000000e+00,2.477032e+06,2.462372e+06,6.000000e+06,2.000000e+06,1.460000e+03,6.516000e+04


In [49]:
df.iloc[:,40:50].describe()

,Pkt Len Mean,Pkt Len Std,Pkt Len Var,FIN Flag Cnt,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count
count,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07
mean,7.762634e+01,1.214681e+02,4.117054e+04,4.795434e-03,4.389543e-02,1.873980e-01,3.920842e-01,3.316020e-01,4.171302e-02,1.638027e-04
std,1.037008e+02,1.625302e+02,2.129773e+05,6.908283e-02,2.048624e-01,3.902307e-01,4.882153e-01,4.707888e-01,1.999326e-01,1.279750e-02
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,5.233333e+01,3.233162e+01,1.045333e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,1.276400e+02,2.874845e+02,8.264735e+04,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
max,1.734498e+04,2.278829e+04,5.190000e+08,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [50]:
df.iloc[:,50:60].describe()

,ECE Flag Cnt,Down/Up Ratio,Pkt Size Avg,Fwd Seg Size Avg,Bwd Seg Size Avg,Subflow Fwd Pkts,Subflow Fwd Byts,Subflow Bwd Pkts,Subflow Bwd Byts,Init Fwd Win Byts
count,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07
mean,1.873986e-01,4.964363e-01,8.954040e+01,5.031274e+01,1.130759e+02,2.353319e+01,9.730373e+02,6.312706e+00,4.730909e+03,8.787202e+03
std,3.902312e-01,9.967273e-01,1.082233e+02,6.050561e+01,1.641056e+02,1.521137e+03,6.216906e+04,1.640256e+02,2.344556e+05,1.623560e+04
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00
50%,0.000000e+00,0.000000e+00,7.209091e+01,3.600000e+01,6.600000e+01,2.000000e+00,4.300000e+01,1.000000e+00,1.010000e+02,9.800000e+02
75%,0.000000e+00,1.000000e+00,1.436316e+02,6.740000e+01,1.625000e+02,5.000000e+00,4.560000e+02,4.000000e+00,5.820000e+02,8.192000e+03
max,1.000000e+00,3.110000e+02,1.747841e+04,1.652931e+04,3.387928e+04,3.096290e+05,1.443918e+08,1.231180e+05,1.563604e+08,6.553500e+04


In [51]:
df.iloc[:,60:70].describe()

,Init Bwd Win Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max
count,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07
mean,8.689131e+03,1.989528e+01,1.799267e+01,1.728776e+05,8.644097e+04,2.623920e+05,1.154657e+05,5.016958e+06,2.866491e+05,5.428144e+06
std,2.061931e+04,1.519917e+03,7.693845e+00,2.505927e+06,1.514575e+06,3.319230e+06,2.114145e+06,2.633178e+08,1.691887e+08,6.253704e+08
min,-1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,-1.000000e+00,0.000000e+00,8.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,-1.000000e+00,0.000000e+00,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,2.300000e+02,1.000000e+00,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,6.553500e+04,3.096280e+05,5.600000e+01,1.140000e+08,7.523241e+07,1.140000e+08,1.140000e+08,3.955714e+11,2.622479e+11,9.797810e+11


In [52]:
df.iloc[:,70:].describe()

,Idle Min,Hour,Minute,Second,Day,Month,Year
count,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07,1.623294e+07
mean,4.685631e+06,6.271300e+00,3.034707e+01,2.940738e+01,1.817051e+01,2.084991e+00,2.018000e+03
std,6.365202e+07,4.062738e+00,1.678201e+01,1.726933e+01,5.765458e+00,2.788720e-01,4.457655e-02
min,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.970000e+03
25%,0.000000e+00,2.000000e+00,1.600000e+01,1.400000e+01,1.600000e+01,2.000000e+00,2.018000e+03
50%,0.000000e+00,6.000000e+00,3.000000e+01,2.900000e+01,2.000000e+01,2.000000e+00,2.018000e+03
75%,0.000000e+00,1.000000e+01,4.600000e+01,4.400000e+01,2.000000e+01,2.000000e+00,2.018000e+03
max,2.399340e+11,1.200000e+01,5.900000e+01,5.900000e+01,2.800000e+01,3.000000e+00,2.018000e+03


In [ ]:
Idle Min: + 11
Idle Mean + 11
Idle Std
Idle Max
Fwd IAT Max
Flow IAT Max

In [53]:
df.dtypes

Label                  int64
Dst Port               int64
Protocol               int64
Flow Duration          int64
Tot Fwd Pkts           int64
Tot Bwd Pkts           int64
TotLen Fwd Pkts      float64
TotLen Bwd Pkts      float64
Fwd Pkt Len Max      float64
Fwd Pkt Len Min      float64
Fwd Pkt Len Mean     float64
Fwd Pkt Len Std      float64
Bwd Pkt Len Max      float64
Bwd Pkt Len Min      float64
Bwd Pkt Len Mean     float64
Bwd Pkt Len Std      float64
Flow Byts/s          float64
Flow Pkts/s          float64
Flow IAT Mean        float64
Flow IAT Std         float64
Flow IAT Max         float64
Flow IAT Min         float64
Fwd IAT Tot          float64
Fwd IAT Mean         float64
Fwd IAT Std          float64
Fwd IAT Max          float64
Fwd IAT Min          float64
Bwd IAT Tot          float64
Bwd IAT Mean         float64
Bwd IAT Std          float64
                      ...   
ACK Flag Cnt           int64
URG Flag Cnt           int64
CWE Flag Count         int64
ECE Flag Cnt  

In [16]:
from sklearn.datasets import dump_svmlight_file

In [17]:
train, validate, test = np.split(df.sample(frac=1, \
                                            random_state=99), \
                                [int(0.7 * len(df)), 
                                int(0.9 * len(df))])

In [31]:
#dump_svmlight_file(X=train.drop('Label', axis=1), \
#                  y=train['Label'], f='train.libsvm')#
#
#dump_svmlight_file(X=validate.drop('Label', axis=1), \
#                  y=validate['Label'], f='validate.libsvm')
#
#dump_svmlight_file(X=test.drop('Label', axis=1), \
#                  y=test['Label'], f='test.libsvm')

In [33]:
##s3.Bucket(bucket).upload_file('train.libsvm', 'train.libsvm')


In [34]:
##s3.Bucket(bucket).upload_file('test.libsvm', 'test.libsvm')

In [35]:

#s3.Bucket(bucket).upload_file('validate.libsvm', 'validate.libsvm')

In [50]:
#boto3.Session().resource('s3').Bucket(bucket).Object('output/' + '/train/train.libsvm').upload_file('train.libsvm')
#boto3.Session().resource('s3').Bucket(bucket).Object('output/' + '/validation/validate.libsvm').upload_file('validate.libsvm')

In [18]:
prefix = 'network-training/output'

In [19]:
dump_svmlight_file(X=train.drop(['Label'], axis=1), y=train['Label'], f='train.libsvm')
dump_svmlight_file(X=validate.drop(['Label'], axis=1), y=validate['Label'], f='validation.libsvm')
dump_svmlight_file(X=test.drop(['Label'], axis=1), y=test['Label'], f='test.libsvm')


In [20]:
from time import gmtime, strftime

In [21]:

boto3.Session().resource('s3').Bucket(bucket).Object(prefix + '/train/train.libsvm').upload_file('train.libsvm')
boto3.Session().resource('s3').Bucket(bucket).Object(prefix + '/validation/validation.libsvm').upload_file('validation.libsvm')

In [25]:
job_name = 'DEMO-network-xgboost-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("Training job", job_name)

from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(boto3.Session().region_name, 'xgboost')

create_training_params = \
{
    "RoleArn": role,
    "TrainingJobName": job_name,
    "AlgorithmSpecification": {
        "TrainingImage": container,
        "TrainingInputMode": "File"
    },
    "ResourceConfig": {
        "InstanceCount": 1,
        "InstanceType": "ml.c4.4xlarge",
        "VolumeSizeInGB": 100
    },
    "InputDataConfig": [
        {
            "ChannelName": "train",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": "s3://{}/{}/train".format(bucket, prefix),
                    "S3DataDistributionType": "FullyReplicated"
                }
            },
            "ContentType": "libsvm",
            "CompressionType": "None"
        },
        {
            "ChannelName": "validation",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": "s3://{}/{}/validation".format(bucket, prefix),
                    "S3DataDistributionType": "FullyReplicated"
                }
            },
            "ContentType": "libsvm",
            "CompressionType": "None"
        }
    ],
    "OutputDataConfig": {
        "S3OutputPath": "s3://{}/{}/xgboost-network/output".format(bucket, prefix)
    },
    "HyperParameters": {
        "max_depth":"3",
        "eta":"0.1",
        "eval_metric":"auc",
        "scale_pos_weight":"2.0",
        "subsample":"0.5",
        "objective":"binary:logistic",
        "num_round":"100"
    },
    "StoppingCondition": {
        "MaxRuntimeInSeconds": 60 * 60
    }
}

Training job DEMO-network-xgboost-2019-02-23-04-36-48


In [26]:
%%time

sm = boto3.client('sagemaker')
sm.create_training_job(**create_training_params)

status = sm.describe_training_job(TrainingJobName=job_name)['TrainingJobStatus']
print(status)

try:
    sm.get_waiter('training_job_completed_or_stopped').wait(TrainingJobName=job_name)
finally:
    status = sm.describe_training_job(TrainingJobName=job_name)['TrainingJobStatus']
    print("Training job ended with status: " + status)
    if status == 'Failed':
        message = sm.describe_training_job(TrainingJobName=job_name)['FailureReason']
        print('Training failed with the following error: {}'.format(message))
        raise Exception('Training job failed')

InProgress
Training job ended with status: Completed
CPU times: user 124 ms, sys: 4.68 ms, total: 129 ms
Wall time: 14min 1s


In [27]:
create_model_response = sm.create_model(
    ModelName=job_name,
    ExecutionRoleArn=role,
    PrimaryContainer={
        'Image': container,
        'ModelDataUrl': sm.describe_training_job(TrainingJobName=job_name)['ModelArtifacts']['S3ModelArtifacts']})

print(create_model_response['ModelArn'])

arn:aws:sagemaker:us-west-2:774023778771:model/demo-network-xgboost-2019-02-23-04-36-48


In [28]:
xgboost_endpoint_config = 'DEMO-network-xgboost-config-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(xgboost_endpoint_config)
create_endpoint_config_response = sm.create_endpoint_config(
    EndpointConfigName=xgboost_endpoint_config,
    ProductionVariants=[{
        'InstanceType': 'ml.t2.medium',
        'InitialInstanceCount': 1,
        'ModelName': job_name,
        'VariantName': 'AllTraffic'}])

print("Endpoint Config Arn: " + create_endpoint_config_response['EndpointConfigArn'])


DEMO-network-xgboost-config-2019-02-23-04-52-07
Endpoint Config Arn: arn:aws:sagemaker:us-west-2:774023778771:endpoint-config/demo-network-xgboost-config-2019-02-23-04-52-07


In [29]:
%%time

xgboost_endpoint = 'DEMO-network-xgboost-endpoint-' + strftime("%Y%m%d%H%M", gmtime())
print(xgboost_endpoint)
create_endpoint_response = sm.create_endpoint(
    EndpointName=xgboost_endpoint,
    EndpointConfigName=xgboost_endpoint_config)
print(create_endpoint_response['EndpointArn'])

resp = sm.describe_endpoint(EndpointName=xgboost_endpoint)
status = resp['EndpointStatus']
print("Status: " + status)

try:
    sm.get_waiter('endpoint_in_service').wait(EndpointName=xgboost_endpoint)
finally:
    resp = sm.describe_endpoint(EndpointName=xgboost_endpoint)
    status = resp['EndpointStatus']
    print("Arn: " + resp['EndpointArn'])
    print("Status: " + status)

    if status != 'InService':
        message = sm.describe_endpoint(EndpointName=xgboost_endpoint)['FailureReason']
        print('Endpoint creation failed with the following error: {}'.format(message))
        raise Exception('Endpoint creation did not succeed')

DEMO-network-xgboost-endpoint-201902230452
arn:aws:sagemaker:us-west-2:774023778771:endpoint/demo-network-xgboost-endpoint-201902230452
Status: Creating
Arn: arn:aws:sagemaker:us-west-2:774023778771:endpoint/demo-network-xgboost-endpoint-201902230452
Status: InService
CPU times: user 142 ms, sys: 20 ms, total: 162 ms
Wall time: 5min 31s


In [30]:
runtime = boto3.client('runtime.sagemaker')


In [31]:
def do_predict(data, endpoint_name, content_type):
    payload = '\n'.join(data)
    response = runtime.invoke_endpoint(EndpointName=endpoint_name, 
                                   ContentType=content_type, 
                                   Body=payload)
    result = response['Body'].read()
    result = result.decode("utf-8")
    result = result.split(',')
    preds = [float((num)) for num in result]
    preds = [round(num) for num in preds]
    return preds

def batch_predict(data, batch_size, endpoint_name, content_type):
    items = len(data)
    arrs = []
    
    for offset in range(0, items, batch_size):
        if offset+batch_size < items:
            results = do_predict(data[offset:(offset+batch_size)], endpoint_name, content_type)
            arrs.extend(results)
        else:
            arrs.extend(do_predict(data[offset:items], endpoint_name, content_type))
        sys.stdout.write('.')
    return(arrs)

In [33]:
import sys

In [34]:
%%time
import json

with open('test.libsvm', 'r') as f:
    payload = f.read().strip()

labels = [int(line.split(' ')[0]) for line in payload.split('\n')]
test_data = [line for line in payload.split('\n')]
preds = batch_predict(test_data, 100, xgboost_endpoint, 'text/x-libsvm')

print ('\nerror rate=%f' % ( sum(1 for i in range(len(preds)) if preds[i]!=labels[i]) /float(len(preds))))

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [35]:
pd.crosstab(index=np.array(labels), columns=np.array(preds))


col_0,0,1
row_0,,
0,1341062,7258
1,10611,264364


In [36]:
sm.delete_endpoint(EndpointName=xgboost_endpoint)

{'ResponseMetadata': {'RequestId': 'ac700f59-4491-442e-bd48-23abc6386463',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'ac700f59-4491-442e-bd48-23abc6386463',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Sat, 23 Feb 2019 05:16:34 GMT'},
  'RetryAttempts': 0}}